# Computational Graphs
*Christoph Heindl 2017, https://github.com/cheind/py-cgraph/*

Computational graphs (CG) are a way of representing mathematical functions and how values propagate between mathematical expressions. Among other things, the study of CG leads to deeper understanding of calculus, especially in simplifying taking derivatives by breaking complex expressions into simpler ones and chaining them up. When implemented in a programming language, CGs are able to automatically differentiate functions numerically exact or even symbolically. 

CGs thus play an important rule in mathematical function optimization, especially when computing derivatives is analytically by hand becomes infeasable. Supervised training of neural networks, for example, maps to optimizing (minimizing) a cost function with respect to all the neural weights in the network chained in different layers. Optimizing is usually performed using a gradient descent approach which requires first order derivatives of the cost function with respect to all the parameters in the network. As we will see, CGs not only make this doable but also provide an computationally efficient algorithm named [backpropagation](https://en.wikipedia.org/wiki/Backpropagation) to compute the partial derivatives.

## What this series will cover

This notebook and the following ones give an introduction into CGs and how to implement them in Python. By the end we will 

 - evaluate mathematical expressions represented as CGs,
 - perform [automatic differentiation](https://en.wikipedia.org/wiki/Automatic_differentiation) to find numerically exact  derivatives (up to floating point precision),
 - perform [symbolic differentiation](https://en.wikipedia.org/wiki/Symbolic_computation) to deduce higher order derivatives,
 - simplify expressions to improve performance and readability.

## What this series will not cover

To keep the code basis readable there are some shortcomings to the developed framework. Foremost it is not complete. That means you won't be able to plugin every possible function and expect it return the correct result. This is mostly a problem of not providing derivatives for all elementary functions. However, the framework is structured in such a way that you will find it easy to add new blocks it and make it even more feature complete.

Also, we'll be mostly dealing with so called multi variate real-valued scalar functions $f:\mathbb {R} ^{n}\to \mathbb {R}$. In other words, we constrain our self to real values and functions that have multiple inputs but only output a single scalar. A glimpse on how to use the framework for vector-valued functions will be given in the examples section towards the end of this document.

If you are looking for production ready libraries you might want to have a look at
 - [SymPy](http://www.sympy.org/en/index.html) for symbolic computation
 - [Autograd](https://github.com/HIPS/autograd) for numeric differentation using numpy

In case you want to stick around and learn something about the fundamental concepts that drive these libraries, your are welcome.

## Introduction to computational graphs (CG)

Consider a function $$f(x,y) := (x + y)x$$

We'll call $x$ and $y$ symbols, $+$ and $*$ will be called operations / functions / nodes. When we evaluate $f$, we first add up the values of $x$ and $y$ and then multiply the result by $x$. The output of the multiplication is what we call the value of $f(x,y)$. Now, a CG is a [directed graph](https://en.wikipedia.org/wiki/Directed_graph) that represents this procedure. A graphical representation of CG for $f$ is shown below

![](images/intro_0.png)

### Computing the value of $f(x,y)$

Computing the value of $f$ in the CG is a matter of following directed edges of the graph. Assume we want to evaluate $f(2,3)$. First, we send 2 and 3 along the out-edges of $x$ and $y$ respectively.

![](images/intro_1.png)

Next, we compute the value of $+$. Note that $*$ cannot be computed as one of its inputs, namely $+$ is missing.

![](images/intro_2.png)

Finally, we find value of $f$ by evaluating $*$

![](images/intro_3.png)

In evaluating the value of $f$ one needs to process all processors of a node $n$ before evaluating $n$ itself. Such an ordering on a CG is called a [topological order](https://en.wikipedia.org/wiki/Topological_sorting). 

### Computing the partial derivatives

We will now turn our attention towards computing of [partial derivatives](https://en.wikipedia.org/wiki/Partial_derivative). In order to do so, we will be a bit more abstract and use symbols for the outputs of all nodes. Like so.

![](images/intro_4.png)

Please take a moment and convince yourself that $f(x,a) = f(x,y)$. 

#### Partial derivatives of isolated nodes

Now consider any node in this CG in isolation, irrespectively of where it is located in the graph. When computing partial derivatives of isolated nodes with respect to their inputs, each input is treated as a separate abstract symbol. Therefore taking the derivative completely ignores the fact that each input symbol might be the result a complex operation by itself. By taking this perspective of isolating nodes, computation of derivatives can be simplified.

For example, take the multiplication node 

$$f(x,a) := ax$$ 

in isolation. Computing the partial derivatives of $f(x,a)$ with respect to its inputs, requires

$$\frac{\mathrm{d}f(x,a)}{\mathrm{d}a}, \frac{\mathrm{d}f(x,a)}{\mathrm{d}x}$$

to be found. Of course this will amount to

$$\frac{\mathrm{d}f(x,a)}{\mathrm{d}a}=x, \frac{\mathrm{d}f(x,a)}{\mathrm{d}x}=a$$

Similarily, for the addition node

$$a(x,y) := x+y$$

we get

$$\frac{\mathrm{d}a(x,y)}{\mathrm{d}x}=1, \frac{\mathrm{d}a(x,y)}{\mathrm{d}y}=1$$

In order to display the partial derivatives in the CG diagrams, we will use backward oriented arrows between nodes to which we attach the corresponding partial derivatives as shown below.

![](images/intro_5.png)

For each node, we consider all of its inputs. For each input arrow we generate a corresponding backward oriented arrow and assign it the partial derivative of the node's output with respect to the input arrow. Note that if a node has multiple inputs and a single output, we call the partial derivatives with respect to all inputs the [gradient](https://en.wikipedia.org/wiki/Gradient).

#### Partial derivatives of nodes with respect to other nodes

To compute the partial derivative for any node $n$ with respect to function $f(x,y):=(x+y)x$, or $f(x,a)$ in the CG diagram, one needs to
 1. Find all the backward pathes starting from $f(x,a)$ reaching $n$.
 1. For each path build the product of partial derivatives along its chain of backward arrows.
 1. Sum over all path products of partial derivatives.
 
For example, the partial derivative of $\frac{\mathrm{d}f(x,y)}{\mathrm{d}y}$ amounts to

$$\frac{\mathrm{d}f(x,y)}{\mathrm{d}y} = 
\frac{\mathrm{d}f(x,a)}{\mathrm{d}f(x,a)}
\frac{\mathrm{d}f(x,a)}{\mathrm{d}a(x,y)}
\frac{\mathrm{d}a(x,y)}{\mathrm{d}y}
$$

There is only one backward oriented path from $f(x,a)$ to $y$, so there is no summation. Note that $f(x,a)$ is equivalent to $f(x,y)$ by construction. By substitution (see CG diagram above) we find

$$\frac{\mathrm{d}f(x,y)}{\mathrm{d}y} = 1*x*1 = x$$

Finding $\frac{\mathrm{d}f(x,y)}{\mathrm{d}x}$ is very similar, except that this time we have two paths in which $x$ can be reached from $f(x,a)$.

$$\frac{\mathrm{d}f(x,y)}{\mathrm{d}x} = 
    \frac{\mathrm{d}f(x,a)}{\mathrm{d}f(x,a)}
    \frac{\mathrm{d}f(x,a)}{\mathrm{d}a(x,y)}
    \frac{\mathrm{d}a(x,y)}{\mathrm{d}x} +
    \frac{\mathrm{d}f(x,a)}{\mathrm{d}f(x,a)}
    \frac{\mathrm{d}f(x,a)}{\mathrm{d}x}   
$$

By subsitution we get

$$\frac{\mathrm{d}f(x,y)}{\mathrm{d}x} = 1*x*1 + 1*a(x,y) = x + (x+y) = 2x+y$$

With these tools you are able to compute the derivatives of 'any' function, no matter how complex it is. Here's what you do:
First, decomposing the function into operations simple enough, so you know how to compute isolated partial derivatives for individual nodes. Next, build a CG based on your decomposition. Apply the 3-step algorithm above to find partial derivatives of any two connected nodes in the CG (usually you do this for the input variables).

### Backpropagation

While you can use the above recipe to compute partial derivatives for any two nodes connected by a path in the CG, it should be noted that it is not computationally efficient yet. Notice the terms 

$$\frac{\mathrm{d}f(x,a)}{\mathrm{d}f(x,a)}\frac{\mathrm{d}f(x,a)}{\mathrm{d}a(x,y)}$$

appear in computations of both $\frac{\mathrm{d}f(x,y)}{\mathrm{d}x}$ and $\frac{\mathrm{d}f(x,y)}{\mathrm{d}y}$. A more clever way to spare redundant computations is this: Turn the backward arrows into a computational graph on its own and apply a topological sorting to it. This is the same procedure as we've seen in evaluting a CG, but this time the order is reversed (as backward edges are used). Traverse the CG, starting from $f(x,a)$ in order and perform the following for each node $n$.
 1. Compute $\frac{\mathrm{d}f(x,a)}{\mathrm{d}n}$ by summing over all incoming backward edges (backward arrows are now node inputs)
 1. Compute the isolated partial derivatives with respect to forward incoming edges $i$ as $\frac{\mathrm{d}n}{\mathrm{d}n_i}$
 1. For each outgoing backward edge compute the product $\frac{\mathrm{d}f(x,a)}{\mathrm{d}n}\frac{\mathrm{d}n}{\mathrm{d}n_i}$.

The above procedure in pictures looks like this. First, we let the value along the very first backward edge be 1. This will allow us to kickstart the procedure above.

![](images/intro_6.png)

Next we process $f(x,a)$

![](images/intro_7.png)

Then we process $a(x,y)$

![](images/intro_8.png)

Finally at $x$ and $y$ we would only perform the first step of the above algorithm to correctly compute $\frac{\mathrm{d}f(x,y)}{\mathrm{d}x}$ and $\frac{\mathrm{d}f(x,y)}{\mathrm{d}y}$

When computing the partial derivatives numerically, one would evaluate the terms at the corresponding values that propagated through the graph in the forward (value) pass. Reconsidering our numeric example from before, were we let $x=2$ and $y=3$ we'd get the following graph

![](images/intro_9.png)

Finally at $x$ and $y$ we'd get $\frac{\mathrm{d}f(x,y)}{\mathrm{d}x}=7$ and $\frac{\mathrm{d}f(x,y)}{\mathrm{d}y}=2$. Mind you that we also know the partial derivatives with respect to $f(x,y)$ for all the intermediate nodes (just the addition in this example). All it took was visiting each node twice, once when computing the node's output value and once in the backward pass.

The chained forward-backward passes as explained above is also known as reverse [automatic differentiation](https://en.wikipedia.org/wiki/Automatic_differentiation), a technique to numerically evaluate the partial derivatives of a function composed of a set of operations for which isolated partial derivatives are known. Note, auto-diff can be executed also in forward mode which won't be covered in this text.

Note that backpropagation in neural network training refers not only to backpropagating partial derivatives of a loss function with respect to neural weights and biases, but also means application of gradient descent methods to update the parameters accordingly, so that the objective function is mininmized. We will come back to that when doing function optimization.

### Summary and Outlook

Up to this point computational graphs (CG) were introduced. A method to compute the value of a CG was presented that used forward propagation of values. Next, a recipe to compute the partial derivatives through backward traversal was given. We've improved the initial formulation by factoring out common terms in what lead to the backpropagation algorithm.

In the [next part](01_Computational_Graphs-Symbolic_Computation.ipynb) a Python library for symbolic and numeric computations will be developed.